In [ ]:
!pip install flask-ngrok
#2N85BWCuT3u7fwU4SftBtvgG9bZ_7tjq6AEs1np3TWgiCkfuK API key for ngrok

In [ ]:
!pip install pyngrok

In [3]:
!ngrok authtoken 2N84c3PX2BWS6WKB7xYGRKnwDUb_N6LnBWeHqYeVGQnwZvbt

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
from flask import Flask,render_template,request
from flask_ngrok import run_with_ngrok
from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import os
from pickle import *
import pickle
import numpy as np
from keras.models import Model, load_model
from keras.layers import Input, Dense, Dropout, LSTM, Embedding
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical, plot_model
from keras.layers import concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
from PIL import Image
from keras.models import load_model

In [5]:
# extract features from each photo in the directory
def extract_feat(filename):
    # load the model
    model = VGG16()
    # re-structure the model
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    # load the photo
    image = load_img(filename, target_size=(224, 224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # get features
    feature = model.predict(image, verbose=0)
    return feature

# map an integer to a word
def word_for_id(integer, tokenizr):
    for word, index in tokenizr.word_index.items():
        if index == integer:
            return word
    return None
 

In [6]:
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'startseq'
    # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
        # convert probability to integer
        yhat = np.argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        # stop if we predict the end of the sequence
        if word == 'endseq':
            break
    return in_text

In [ ]:
from PIL import Image
from keras.models import load_model
# load the model
modl = load_model('/content/drive/MyDrive/Colab Notebooks/latest20epochs.h5')
image_dataset_path = '/content/drive/MyDrive/Colab Notebooks/Flickr8k_Dataset/Flicker8k_Dataset'
caption_dataset_path = '/content/drive/MyDrive/Colab Notebooks/Flickr8k_Dataset/Flickr8k_text/Flickr8k.token.txt'
obj1=open('/content/drive/MyDrive/Colab Notebooks/new_caption_dict.pkl','rb')
new_captions_dict=pickle.load(obj1)
obj2=open('/content/drive/MyDrive/Colab Notebooks/train_validate_images.pkl','rb')
train_validate_images=pickle.load(obj2)


# generate description
tokenizr = Tokenizer()
tokenizr.fit_on_texts([caption for image, caption in new_captions_dict.items() if image in train_validate_images])
max_length = 30

for count in range(10):

    photo = extract_feat('{}.jpg'.format(image_dataset_path+'/'+train_validate_images[count]))  
    im=Image.open('{}.jpg'.format(image_dataset_path+'/'+train_validate_images[count]))
    plt.imshow(im)
    plt.show()
    # generate description
    description = generate_desc(modl, tokenizr, photo, max_length)
    print("Predicted caption -> ", description)
    print()
    print("Actual caption -> ", new_captions_dict[train_validate_images[count]])
    print('*********************************************************************')
    print()

Output hidden; open in https://colab.research.google.com to view.

In [7]:
# load the model
modl = load_model('/content/drive/MyDrive/Colab Notebooks/latest20epochs.h5')
image_dataset_path = '/content/drive/MyDrive/Colab Notebooks/Flickr8k_Dataset/Flicker8k_Dataset'
caption_dataset_path = '/content/drive/MyDrive/Colab Notebooks/Flickr8k_Dataset/Flickr8k_text/Flickr8k.token.txt'
obj1=open('/content/drive/MyDrive/Colab Notebooks/new_caption_dict.pkl','rb')
new_captions_dict=pickle.load(obj1)
obj2=open('/content/drive/MyDrive/Colab Notebooks/train_validate_images.pkl','rb')
train_validate_images=pickle.load(obj2)
tokenizr = Tokenizer()
tokenizr.fit_on_texts([caption for image, caption in new_captions_dict.items() if image in train_validate_images])
max_length = 30 
photo = extract_feat('/content/drive/MyDrive/Colab Notebooks/Flickr8k_Dataset/Flicker8k_Dataset/1000268201_693b08cb0e.jpg')
description = generate_desc(modl, tokenizr, photo, max_length)
print("Predicted caption -> ", description)

553467096/553467096 [==============================] - 3s 0us/step
Predicted caption ->  startseq man in blue jacket and red hat holds red guitar closed endseq


In [ ]:
app=Flask(__name__,template_folder='/content/drive/MyDrive/Colab Notebooks/vcs_app_flask/templates',static_folder='/content/drive/MyDrive/Colab Notebooks/vcs_app_flask/static')
app.config['SEND_FILE_MAX_AGE_DEFAULT']=1
run_with_ngrok(app)


@app.route('/')
def index():

  return render_template('index.html')
  
@app.route('/after',methods=['GET','POST'])
def after():
  imagefile = request.files['file1']
  if not imagefile:
    return "Image not submitted!",404
  imagefile.save('/content/drive/MyDrive/Colab Notebooks/vcs_app_flask/static/file1.jpg')
  photo = extract_feat('/content/drive/MyDrive/Colab Notebooks/vcs_app_flask/static/file1.jpg')
  description = generate_desc(modl, tokenizr, photo, max_length)

  print("Predicted caption -> ", description)

  return render_template('predict.html',caption=description)

if __name__=="__main__":
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://ae99-35-224-110-178.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [31/Mar/2023 07:00:19] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Mar/2023 07:00:20] "GET /favicon.ico HTTP/1.1" 404 -


Predicted caption ->  startseq boy in blue shirt is riding bike down the concrete hill endseq


INFO:werkzeug:127.0.0.1 - - [31/Mar/2023 07:00:56] "POST /after HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Mar/2023 07:00:56] "GET /static/file1.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Mar/2023 07:01:33] "POST /after HTTP/1.1" 200 -


Predicted caption ->  startseq group of people are standing in front of italian company endseq


INFO:werkzeug:127.0.0.1 - - [31/Mar/2023 07:01:34] "GET /static/file1.jpg HTTP/1.1" 200 -
